### Задание на Семинар 6

**НЕ изменяйте и не удаляйте ячейки, написанные с помощью markdown, только ячейки с кодом**

Справочные материалы:
* [шпаргалка по Pandas](http://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
* [Отладчик регулярных выражений](https://regex101.com)

_Примечание: при выполнении задания можно пользоваться библиотеками BeautifulSoup и RE как угодно_

* Создайте датафрейм data с колонками (в скобках - название колонки в датафрейме):
 * Идентификатор (id)
 * Авторы (authors)
 * Название (title)
 * Новая цена (new_price)
 
 (**1 балл**)

In [ ]:
import bs4
import json
import pandas as pd
import requests
import re

data = pd.DataFrame(columns = ['id', 'authors', 'title', 'new_price'])
data

,id,authors,title,new_price


* Откройте сайт [Буквоеда](https://www.bookvoed.ru/books?genre=26&view=list)
* Заполните поля датафрейма данными из списка (книги в карусели пропустите)

(**1 балл** за каждый столбец)

In [ ]:
url = 'https://www.bookvoed.ru/books?genre=26&view=list'
soup = bs4.BeautifulSoup(requests.get(url).text)
books = soup.find_all('div', class_='Pc')

def parse(book):
    a = book.find('h5').find('a')
    _id = list(json.loads(a.attrs.get('data-gtm_book_click_detail'))['b'].keys())[0]
    title = a.text
    price = re.sub('(.*?)\n', '',book.find('div', class_='l ed').text.strip()).replace('руб.', ' ')
    authors = book.find('h5', class_='ld').text
    return pd.Series({'id': _id, 'authors': authors, 'title': title, 'new_price': price})

data = pd.DataFrame(list(map(parse, books)))
data.head()

,authors,id,new_price,title
0,Гэлбрейт Роберт,9391987,569,Смертельная белизна: роман
1,Рэнд Айн,662185,995,Атлант расправил плечи. В 3 книгах.
2,Акунин Борис,7909666,589,Не прощаюсь. Приключения Эраста Фандорина в ХХ...
3,Хэнкс Том Джеффри,7871611,384,Уникальный экземпляр. Истории о том о сём
4,Леви Марк,8704004,309,"Не такая, как все : роман"


* Создайте датафрейм shops с колонками (в скобках - название колонки в датафрейме):
 * Идентификатор книги (id)
 * Метро (subway)
 * Краткий адрес (location)
 * Наличие на складе (stock)
 
 (**1 балл**)

In [ ]:
shops = pd.DataFrame(columns = ['id', 'subway', 'location', 'stock'])
shops

,id,subway,location,stock


* Заполните поля датафрейма shops данными о книгах из датафрейма data 

(**1 балл** за каждый столбец)

In [ ]:
def check_shops(book_id):
    soup = bs4.BeautifulSoup(requests.get(f'https://www.bookvoed.ru/book?id={book_id}').text)
    shops = soup.find_all('div', class_='CK')
    
    def parse_shop(shop):
        stock = shop.find('div', class_='GK').text.strip()
        subway = shop.find('div', class_ = 'WE')
        location = shop.find('a', class_='VE').text
        return pd.Series({'id': book_id, 'stock': stock, 'subway': subway.text if subway else '', 'location': location})
    
    return pd.DataFrame(list(map(parse_shop, shops)))
    
shops = pd.concat([check_shops(x) for x in data['id']])
shops